# Coursework 2



In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

def monte_carlo(μ=0, σ=100, price=100, days=252, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    μ = μ / 100
    σ = σ / 100
    Z = rng.normal(loc=1+μ, scale=σ, size=days)
    P = np.empty(days)
    for i in range(days):
        P[i] = price
        price = price * Z[i]
    return P